In [88]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import tensorflow as tf

In [89]:
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(2)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [90]:
 # Drop the non-beneficial EIN column
 app_clean_df = application_df.drop(columns=["EIN"])
 app_clean_df.head(2)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [91]:
# display the number of unique values
app_clean_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [92]:
# # Can we drop the STATUS column?
app_clean_df["STATUS"].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [93]:
# # Can we drop the SPECIAL_CONSIDERATIONS column?
app_clean_df["SPECIAL_CONSIDERATIONS"].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [94]:
# there isn't much variation in the STATUS and SPECIAL_CONSIDERATIONS, we can drop these columns as they have no purpose for our work
app_clean_df = app_clean_df.drop(columns=["STATUS", "SPECIAL_CONSIDERATIONS"])
app_clean_df.head(2)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1


In [95]:
# Look at NAME value counts for binning
name_counts = app_clean_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [96]:
name_counts[name_counts > 5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [97]:
# Determine which values to replace if counts are less than or equal to 5.
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for name in names_to_replace:
  app_clean_df['NAME'] = app_clean_df['NAME'].replace(name, 'Other')

# Check to make sure binning was successful
app_clean_df['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [98]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = app_clean_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [99]:
# Determine which values to replace if application counts are less than 500
app_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app_type in app_types_to_replace:
  app_clean_df['APPLICATION_TYPE'] = app_clean_df['APPLICATION_TYPE'].replace(
      app_type, 'Other'
  )

# Check to make sure binning was successful
app_clean_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [100]:
# Look at CLASSIFICATION value counts for binning
class_counts = app_clean_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [101]:
# Determine which values to replace if counts are less than 1000
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    app_clean_df['CLASSIFICATION'] = app_clean_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
app_clean_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [102]:
# Generate our categorical variable lists
cat_vars = list(app_clean_df.dtypes[app_clean_df.dtypes == 'object'].index)
cat_vars

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [103]:
# perform one-hot encoding on the data
app_clean_df = pd.get_dummies(app_clean_df)
app_clean_df.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALPHA PHI OMEGA,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [104]:
# Split our preprocessed data into our features and target arrays
x = app_clean_df.drop(columns=['IS_SUCCESSFUL']).values
y = app_clean_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [105]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [106]:
# Create a sequential model
nn_model= tf.keras.models.Sequential()

In [107]:
# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=100, input_dim = 395, activation ="relu"))

In [108]:
# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation ="sigmoid"))

In [109]:
# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation ="sigmoid"))

In [110]:
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))



In [111]:
# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 100)               39600     
                                                                 
 dense_14 (Dense)            (None, 30)                3030      
                                                                 
 dense_15 (Dense)            (None, 10)                310       
                                                                 
 dense_16 (Dense)            (None, 1)                 11        
                                                                 
Total params: 42951 (167.78 KB)
Trainable params: 42951 (167.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [112]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [113]:
len(x_train[0])

395

In [114]:
# Train the model
fit_model = nn_model.fit(x_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.4876 - accuracy: 0.7765
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.4446 - accuracy: 0.7916
Epoch 3/200
804/804 [==============================] - 2s 3ms/step - loss: 0.4366 - accuracy: 0.7950
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.4321 - accuracy: 0.7958
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.4299 - accuracy: 0.7947
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.4276 - accuracy: 0.7955
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.4258 - accuracy: 0.7971
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.4249 - accuracy: 0.7977
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.4233 - accuracy: 0.7978
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.4226 - accura

In [115]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4466 - accuracy: 0.7960 - 551ms/epoch - 2ms/step
Loss: 0.44659531116485596, Accuracy: 0.7960349917411804


In [116]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [117]:
# Create a random forest classifier.
rfc = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rfc = rfc.fit(x_train_scaled, y_train)

# Evaluate the model
y_predicted = rfc.predict(x_test_scaled)
print(f'Random Forest accuracy: {accuracy_score(y_test, y_predicted)*100:.2f}')

Random Forest accuracy: 78.22
